In [20]:
!pip install agentpy

# Descripción del sistema multiagente
Esta simulación urbana en AgentPy modela un cruce de intersección (crossroad) donde conviven peatones y vehículos en una cuadrícula. Se definen las aceras, carreteras y múltiples pasos peatonales, con obstáculos aleatorios en banquetas. Los peatones usan el algoritmo Weighted A* para planear rutas desde un origen aleatorio hasta un destino común, evitando obstáculos y el tráfico. Los vehículos circulan por las calles de forma lineal, reapareciendo al salir del área hasta cumplir con un número definido.

# Que queda pendiente?


*   **~~Destino de los peatones~~** : Por ahora todos los agentes peatones se dirigen al mismo lugar para facilitar la implementación, pero para la versión final, lo ideal sería que los agentes tengan diferentes destinos.

*   **Implementación del "Agente Autoridad"** : El agente autoridad, simplemente se dedicara a remover ciertos obstaculos como los comercios informales y retornara a un mismo punto una vez lo haya logrado.

*   **~~Implementación de un sistema de semaforos~~** : Para alcanzar un nivel de realismo mas acertado, se buscara la implementación de un sistema de semaforos que indicara a los peatones cuando cruzar y cuando no hacerlo, adicionalmente, se incrementara la cantidad de vehiculos circulando que también reaccionaran ante el semaforo.

* **~~Atributo: Peatones arriesgados~~** : La cantidad de obstaculos en las banquetas provocara que ciertos agentes tomen decisiones mas arriesgadas como bajar al arroyo.





In [23]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import json
import socket
import time

class NetworkSender:
    def __init__(self, host='127.0.0.1', port=1116):
        self.host = host
        self.port = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connected = False
        
    def connect(self):
        try:
            self.socket.connect((self.host, self.port))
            self.connected = True
            print("Connected to Unity server")
        except Exception as e:
            print(f"Could not connect to Unity server: {e}")
            self.connected = False
    
    def send_data(self, data):
        if not self.connected:
            self.connect()
            if not self.connected:
                return
                
        try:
            json_data = json.dumps(data) + "$"  # Add delimiter
            self.socket.sendall(json_data.encode('utf-8'))
        except Exception as e:
            print(f"Error sending data: {e}")
            self.connected = False
    
    def close(self):
        self.socket.close()
        self.connected = False

def weighted_a_star(grid, start, goal, weight=1.3, max_attempts=7):
    def heuristic(a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    for attempt in range(max_attempts):
        def neighbors(pos):
            directions = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]
            for d in directions:
                nr, nc = pos[0]+d[0], pos[1]+d[1]
                if 0 <= nr < grid.shape[0] and 0 <= nc < grid.shape[1]:
                    if attempt > 2 and grid[nr, nc] == 1 and np.random.rand() < 0.5:
                        continue
                    yield (nr, nc)

        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: weight * heuristic(start, goal)}

        while open_set:
            current = min(open_set, key=lambda pos: f_score.get(pos, np.inf))
            if current == goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(start)
                return path[::-1]

            open_set.remove(current)
            for neighbor in neighbors(current):
                if grid[neighbor] in (1, 2, 4) and not (attempt > 2 and grid[neighbor] == 1 and np.random.rand() < 0.5):
                    continue

                tentative_g = g_score[current] + (1.4 if neighbor[0] != current[0] and neighbor[1] != current[1] else 1)
                if tentative_g < g_score.get(neighbor, np.inf):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + weight * heuristic(neighbor, goal)
                    open_set.add(neighbor)

    return [start, goal]

class Authority(ap.Agent):
    def setup(self):
        self.pos = self.model.find_empty_spot()
        self.completed = False
        self.path = []
        self.target_obstacle = None
        self.removed_obstacles = 0
        self.search_radius = 10
        self.last_positions = []
        self.stuck_count = 0
        self.speed = 2
        self.current_target = None
        self.type = "authority"
        self.id = f"auth_{self.id}"

    def move(self):
        if self.completed or self.model.obstacle_count == 0:
            self.completed = True
            return

        if self.model.grid[self.pos] == 1:
            self._remove_obstacle()
            return

        if self.current_target is None or (self.pos == self.current_target and self.model.grid[self.pos] != 1):
            self._find_new_target()
            if self.current_target is None:
                self.completed = True
                return

        if not self.path or self.stuck_count > 2:
            self.path = self._calculate_path_to(self.current_target)
            if not self.path:
                self._find_new_target()
                return

        if self.path:
            moves_left = self.speed
            while moves_left > 0 and self.path:
                next_pos = self.path[0]
                
                if not self.model.is_valid_authority_move(self.pos, next_pos):
                    self.path = []
                    self.stuck_count += 1
                    break
                
                self.pos = next_pos
                self.path.pop(0)
                moves_left -= 1
                self.stuck_count = 0

                if self.pos == self.current_target:
                    if self.model.grid[self.pos] == 1:
                        self._remove_obstacle()
                    else:
                        self._find_new_target()
                    break

    def _remove_obstacle(self):
        if self.model.grid[self.pos] == 1:
            self.model.grid[self.pos] = 0
            self.model.obstacle_count -= 1
            self.removed_obstacles += 1
            self.path = []
            self.current_target = None
            self.stuck_count = 0

    def _find_new_target(self):
        min_dist = float('inf')
        nearest_obstacle = None
        
        if self.model.obstacle_count < 10:
            search_range = [(r,c) for r in range(self.model.height) 
                           for c in range(self.model.width) 
                           if self.model.grid[r,c] == 1]
        else:
            search_range = [(r,c) for r in range(max(0, self.pos[0]-self.search_radius),
                                   min(self.model.height, self.pos[0]+self.search_radius+1))
                           for c in range(max(0, self.pos[1]-self.search_radius),
                                       min(self.model.width, self.pos[1]+self.search_radius+1))
                           if self.model.grid[r,c] == 1]

        for obstacle in search_range:
            dist = abs(obstacle[0] - self.pos[0]) + abs(obstacle[1] - self.pos[1])
            if dist < min_dist:
                min_dist = dist
                nearest_obstacle = obstacle

        if nearest_obstacle:
            self.current_target = nearest_obstacle
            self.path = self._calculate_path_to(nearest_obstacle)
            self.stuck_count = 0
        else:
            self.search_radius = min(self.search_radius + 10, 
                                    max(self.model.width, self.model.height))
            if self.search_radius >= max(self.model.width, self.model.height):
                self.completed = True

    def _calculate_path_to(self, target):
        path = weighted_a_star(
            self.model.grid,
            self.pos,
            target,
            weight=1.3,
            max_attempts=5
        )
        return path if len(path) > 1 else []

class Pedestrian(ap.Agent):
    def setup_agent(self):
        self.pos = self.origin
        self.destination = self.select_farthest_destination()
        self.path = self.calculate_new_path()
        self.step_index = 1
        self.completed = False
        self.waiting_since = 0
        self.failed_attempts = 0
        self.last_positions = []
        self.type = "pedestrian"
        self.id = f"ped_{self.id}"
        self.color = "#0000FF"  # Blue

    def select_farthest_destination(self):
        max_distance = -1
        chosen_dest = None
        for dest in self.model.destinations:
            distance = abs(self.origin[0] - dest[0]) + abs(self.origin[1] - dest[1])
            if distance > max_distance:
                max_distance = distance
                chosen_dest = dest
        return chosen_dest

    def calculate_new_path(self):
        path = weighted_a_star(self.model.grid, self.pos, self.destination,
                             weight=self.model.weight, max_attempts=7)
        return path if len(path) > 1 else [self.pos, self.destination]

    def next_position(self):
        if self.completed:
            return None
        if self.step_index >= len(self.path):
            self.handle_completion()
            return None
        return self.path[self.step_index]

    def move_to(self, next_pos):
        self.pos = next_pos
        self.step_index += 1
        self.waiting_since = 0
        self.failed_attempts = 0
        self.last_positions.append(next_pos)
        if len(self.last_positions) > 5:
            self.last_positions.pop(0)

    def find_alternative_move(self, occupied):
        directions = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]
        possible_moves = []

        for dy, dx in directions:
            nr, nc = self.pos[0] + dy, self.pos[1] + dx
            if (0 <= nr < self.model.height and 0 <= nc < self.model.width and
                self.model.is_valid_pedestrian_move(self.pos, (nr, nc))):

                vehicle_blocking = any(
                    v.pos == (nr, nc) and v.active
                    for v in self.model.vehicles
                )
                authority_blocking = self.model.authority.pos == (nr, nc) and not self.model.authority.completed

                if not vehicle_blocking and not authority_blocking:
                    possible_moves.append((nr, nc))

        if possible_moves:
            best_move = min(
                possible_moves,
                key=lambda p: abs(p[0]-self.destination[0]) + abs(p[1]-self.destination[1])
            )

            if best_move in self.last_positions[-3:]:
                self.path = self.calculate_new_path()
                self.step_index = 1
            else:
                self.pos = best_move
                self.path = self.calculate_new_path()
                self.step_index = 1
        else:
            self.waiting_since += 1
            self.failed_attempts += 1

            if self.failed_attempts > 2 or self.waiting_since > 5:
                self.path = self.calculate_new_path()
                self.step_index = 1
                self.failed_attempts = 0
                self.waiting_since = 0

    def handle_completion(self):
        distance = abs(self.pos[0] - self.destination[0]) + abs(self.pos[1] - self.destination[1])
        if distance <= 1:
            self.completed = True
            if self.pos in self.model.occupied_sidewalk:
                self.model.occupied_sidewalk.discard(self.pos)
            self.model.completed_pedestrians += 1

            if self.model.completed_pedestrians < self.model.max_pedestrians:
                self.reset_for_new_route()
        else:
            self.path = self.calculate_new_path()
            self.step_index = 1

    def reset_for_new_route(self):
        origin = self.model.sample_pedestrian_origin()
        self.origin = origin
        self.pos = origin
        self.model.occupied_sidewalk.add(origin)
        self.destination = self.select_farthest_destination()
        self.path = self.calculate_new_path()
        self.step_index = 1
        self.completed = False
        self.waiting_since = 0
        self.failed_attempts = 0
        self.last_positions = []

class Vehicle(ap.Agent):
    def setup_agent(self):
        (sr, sc), direction = self.model.random_start_and_direction()
        self.start_row, self.start_col = sr, sc
        self.direction = direction
        self.wait_steps = self.model.random.randint(0, 3)
        self.active = False
        self.row = None
        self.col = None
        self.pos = None
        self.completed = False
        self.in_intersection = False
        self.wait_time = 0
        self.type = "vehicle"
        self.id = f"veh_{self.id}"
        self.color = "#FF0000"  # Red

    def next_position(self):
        if not self.active or self.completed or self.pos is None:
            return None
        return (self.row + self.direction[0], self.col + self.direction[1])

    def step(self):
        if self.completed:
            return

        if not self.active:
            self.wait_steps -= 1
            if self.wait_steps <= 0:
                self.row, self.col = self.start_row, self.start_col
                self.pos = (self.row, self.col)
                self.active = True
            return

        next_pos = self.next_position()
        if next_pos and 0 <= next_pos[0] < self.model.height and 0 <= next_pos[1] < self.model.width:
            if self.model.grid[next_pos] in (2, 3):
                self.row, self.col = next_pos
                self.pos = (self.row, self.col)
                self.wait_time = 0
                if (self.pos[0] in self.model.horizontal_road_rows and
                    self.pos[1] in self.model.vertical_road_cols):
                    self.in_intersection = True
                else:
                    self.in_intersection = False
            else:
                self.wait_time += 1
        else:
            self.completed = True

class CrossroadModel(ap.Model):
    def setup(self):
        # Initialize network sender
        self.network_sender = NetworkSender()
        
        self.width = self.p.width
        self.height = self.p.height
        self.obs_density = self.p.obs_density
        self.weight = self.p.weight
        self.max_pedestrians = self.p.max_pedestrians
        self.max_vehicles = self.p.max_vehicles
        self.green_duration = self.p.green_duration
        self.building_size = self.p.building_size

        self.grid = np.zeros((self.height, self.width), dtype=int)
        road_width = 4
        half_v = (self.height - road_width) // 2
        half_h = (self.width - road_width) // 2
        self.horizontal_road_rows = list(range(half_v, half_v + road_width))
        self.vertical_road_cols = list(range(half_h, half_h + road_width))

        # Create roads
        for r in self.horizontal_road_rows:
            self.grid[r, :] = 2
        for c in self.vertical_road_cols:
            self.grid[:, c] = 2

        # Create crosswalks
        crosswalk_horizontal_cols = [self.width // 4, self.width // 2, 3 * self.width // 4]
        crosswalk_vertical_rows = [self.height // 4, self.height // 2, 3 * self.height // 4]
        for col in crosswalk_horizontal_cols:
            for r in self.horizontal_road_rows:
                self.grid[r, col] = 3
        for row in crosswalk_vertical_rows:
            for c in self.vertical_road_cols:
                self.grid[row, c] = 3

        # Destinations
        self.destinations = [
            (15, 24), (15, 1), (0, 9), (2, 17)
        ]
        for dest in self.destinations:
            r, c = dest
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < self.height and 0 <= nc < self.width and self.grid[nr, nc] == 0:
                        self.grid[nr, nc] = 0

        # Sidewalks
        self.sidewalk_north = [(r, c) for r in range(0, half_v) for c in range(self.width)
                             if c not in self.vertical_road_cols and self.grid[r, c] == 0]
        self.sidewalk_south = [(r, c) for r in range(half_v + road_width, self.height)
                             for c in range(self.width) if c not in self.vertical_road_cols and self.grid[r, c] == 0]
        self.sidewalk_west = [(r, c) for r in range(self.height)
                            for c in range(0, half_h) if r not in self.horizontal_road_rows and self.grid[r, c] == 0]
        self.sidewalk_east = [(r, c) for r in range(self.height)
                            for c in range(half_h + road_width, self.width) if r not in self.horizontal_road_rows and self.grid[r, c] == 0]

        # Create obstacles
        self.obstacle_count = 0
        for r, c in self.sidewalk_north + self.sidewalk_south + self.sidewalk_west + self.sidewalk_east:
            if np.random.rand() < self.obs_density:
                self.grid[r, c] = 1
                self.obstacle_count += 1

        # Create buildings
        bs = self.building_size
        building_coords = [
            (0, 0, bs, bs), (0, self.width - bs, bs, self.width),
            (self.height - bs, 0, self.height, bs), (self.height - bs, self.width - bs, self.height, self.width)
        ]
        for r_start, c_start, r_end, c_end in building_coords:
            for r in range(r_start, r_end):
                for c in range(c_start, c_end):
                    if 0 <= r < self.height and 0 <= c < self.width:
                        self.grid[r, c] = 4

        # Initialize agents
        self.occupied_sidewalk = set()
        self.pedestrians = ap.AgentList(self, self.max_pedestrians, Pedestrian)
        self.vehicles = ap.AgentList(self, 0, Vehicle)
        self.authority = Authority(self)
        self.completed_pedestrians = 0
        self.completed_vehicles = 0

        for ped in self.pedestrians:
            origin = self.sample_pedestrian_origin()
            ped.origin = origin
            self.occupied_sidewalk.add(origin)
            ped.setup_agent()

        self.authority.setup()
        
        # Send initial state to Unity
        self.send_state_to_unity()

    def send_state_to_unity(self):
        # Collect all agents data
        agents_data = []
        
        # Add authority
        if hasattr(self, 'authority') and not self.authority.completed:
            agents_data.append({
                "id": self.authority.id,
                "type": self.authority.type,
                "position": {"x": float(self.authority.pos[1]), "y": float(self.authority.pos[0])},
                "color": "#00FF00"  # Green
            })
        
        # Add pedestrians
        for ped in self.pedestrians:
            if not ped.completed:
                agents_data.append({
                    "id": ped.id,
                    "type": ped.type,
                    "position": {"x": float(ped.pos[1]), "y": float(ped.pos[0])},
                    "color": ped.color
                })
        
        # Add vehicles
        for veh in self.vehicles:
            if veh.active and not veh.completed:
                agents_data.append({
                    "id": veh.id,
                    "type": veh.type,
                    "position": {"x": float(veh.pos[1]), "y": float(veh.pos[0])},
                    "color": veh.color
                })
        
        # Add obstacles (static objects)
        for r in range(self.height):
            for c in range(self.width):
                if self.grid[r, c] == 1:  # Obstacle
                    agents_data.append({
                        "id": f"obs_{r}_{c}",
                        "type": "obstacle",
                        "position": {"x": float(c), "y": float(r)},
                        "color": "#A0522D"  # Brown
                    })
                elif self.grid[r, c] == 4:  # Building
                    agents_data.append({
                        "id": f"bld_{r}_{c}",
                        "type": "building",
                        "position": {"x": float(c), "y": float(r)},
                        "color": "#FFA500"  # Orange
                    })
        
        # Create simulation data
        sim_data = {
            "agents": agents_data,
            "step": self.t,
            "trafficLightState": "horizontal_green" if self.is_horizontal_green() else "vertical_green",
            "obstaclesRemoved": self.authority.removed_obstacles if hasattr(self, 'authority') else 0
        }
        
        # Send to Unity
        self.network_sender.send_data(sim_data)

    def find_empty_spot(self):
        empty_spots = [(r,c) for r in range(self.height)
                      for c in range(self.width) if self.grid[r,c] == 0]
        return self.random.choice(empty_spots) if empty_spots else (0, 0)

    def is_valid_authority_move(self, current_pos, next_pos):
        if not (0 <= next_pos[0] < self.height and 0 <= next_pos[1] < self.width):
            return False
        return (self.grid[next_pos] != 4 and 
                not (next_pos[0] in self.horizontal_road_rows and 
                     next_pos[1] in self.vertical_road_cols))

    def sample_pedestrian_origin(self):
        candidates = [pos for pos in (self.sidewalk_north + self.sidewalk_south + self.sidewalk_west + self.sidewalk_east)
                     if pos not in self.occupied_sidewalk]
        return self.random.choice(candidates if candidates else [pos for pos in (self.sidewalk_north + self.sidewalk_south + self.sidewalk_west + self.sidewalk_east) if self.grid[pos] == 0])

    def random_start_and_direction(self):
        choice = self.random.randint(0, 7)
        if choice in [0, 1]:
            row = self.horizontal_road_rows[choice]
            start = (row, 0)
            direction = (0, 1)
        elif choice in [2, 3]:
            row = self.horizontal_road_rows[choice]
            start = (row, self.width - 1)
            direction = (0, -1)
        elif choice in [4, 5]:
            col = self.vertical_road_cols[choice - 4]
            start = (0, col)
            direction = (1, 0)
        else:
            col = self.vertical_road_cols[choice - 4]
            start = (self.height - 1, col)
            direction = (-1, 0)
        return start, direction

    def is_horizontal_green(self):
        cycle = 2 * self.green_duration
        return (self.t % cycle) < self.green_duration

    def step(self):
        # Give priority to authority
        if not self.authority.completed:
            self.authority.move()

        horizontal_green = self.is_horizontal_green()
        self.move_vehicles(horizontal_green)
        self.move_pedestrians()
        
        # Send updated state to Unity after each step
        self.send_state_to_unity()
        
        # Small delay to allow Unity to process
        time.sleep(0.05)

    def move_vehicles(self, horizontal_green):
        occupied = set(p.pos for p in self.pedestrians if not p.completed)
        occupied |= set(v.pos for v in self.vehicles if v.active and not v.completed)
        occupied.add(self.authority.pos)

        for veh in list(self.vehicles):
            if veh.completed:
                self.vehicles.remove(veh)
                self.completed_vehicles += 1
                continue

            next_pos = veh.next_position()
            if next_pos is None:
                veh.step()
                continue

            in_inter = veh.in_intersection
            entering = (not in_inter) and (next_pos[0] in self.horizontal_road_rows and next_pos[1] in self.vertical_road_cols)
            allow = False

            if in_inter:
                allow = True
            elif entering:
                r, c = veh.pos
                if r in self.horizontal_road_rows and horizontal_green:
                    allow = True
                elif c in self.vertical_road_cols and not horizontal_green:
                    allow = True
            else:
                allow = True

            if allow and 0 <= next_pos[0] < self.height and 0 <= next_pos[1] < self.width and next_pos not in occupied and self.grid[next_pos] in (2, 3):
                occupied.discard(veh.pos)
                veh.step()
                if entering:
                    veh.in_intersection = True
                if veh.in_intersection and not (veh.pos[0] in self.horizontal_road_rows and veh.pos[1] in self.vertical_road_cols):
                    veh.in_intersection = False
                occupied.add(veh.pos)
            else:
                veh.wait_time += 1
                if veh.wait_time > 3:
                    veh.in_intersection = False

        while len(self.vehicles) < min(self.max_vehicles, self.t // 10 + 5):
            new_veh = Vehicle(self)
            new_veh.setup_agent()
            self.vehicles.append(new_veh)

    def move_pedestrians(self):
        occupied = set(v.pos for v in self.vehicles if v.active and not v.completed)
        occupied |= set(p.pos for p in self.pedestrians if not p.completed)
        occupied.add(self.authority.pos)

        sorted_peds = sorted(
            [p for p in self.pedestrians if not p.completed],
            key=lambda p: (p.waiting_since, -len(p.path)),
            reverse=True
        )

        for ped in sorted_peds:
            next_pos = ped.next_position()
            if next_pos is None:
                continue

            vehicle_blocking = any(v.pos == next_pos and v.active for v in self.vehicles)
            authority_blocking = self.authority.pos == next_pos and not self.authority.completed

            if not vehicle_blocking and not authority_blocking and self.is_valid_pedestrian_move(ped.pos, next_pos):
                if self.grid[ped.pos] == 0:
                    self.occupied_sidewalk.discard(ped.pos)
                ped.move_to(next_pos)
                if self.grid[next_pos] == 0:
                    self.occupied_sidewalk.add(next_pos)
            else:
                ped.find_alternative_move(occupied)

    def is_valid_pedestrian_move(self, current_pos, next_pos):
        if not (0 <= next_pos[0] < self.height and 0 <= next_pos[1] < self.width):
            return False
        if next_pos in self.destinations:
            return True
        if (next_pos[0] in self.horizontal_road_rows and next_pos[1] in self.vertical_road_cols):
            return False
        return self.grid[next_pos] in (0, 2, 3)

def plot(model, ax):
    ax.clear()
    cmap = {0: 'lightgray', 1: 'sienna', 2: 'gray', 3: 'white', 4: 'orange'}
    for r in range(model.height):
        for c in range(model.width):
            ax.add_patch(plt.Rectangle((c, model.height - 1 - r), 1, 1,
                         color=cmap.get(model.grid[r, c], 'black'), ec='black', lw=0.1))

    destinations_markers = ['*', 'P', 'X', 'D']
    destinations_colors = ['gold', 'green', 'purple', 'cyan']
    for i, (dr, dc) in enumerate(model.destinations):
        ax.scatter(dc + 0.5, model.height - 1 - dr + 0.5,
                  c=destinations_colors[i], s=200, marker=destinations_markers[i],
                  edgecolors='black', linewidth=1,
                  label=f'Destino {i+1}', zorder=10)

    horizontal_green = model.is_horizontal_green()
    status = 'HORIZONTALES VERDE' if horizontal_green else 'VERTICALES VERDE'
    ax.set_title(f"Paso: {model.t} - {status} - Obstáculos: {model.obstacle_count} - Autoridad: {'Activa' if not model.authority.completed else 'Inactiva'} ({model.authority.removed_obstacles} removidos)")

    ped_x = [p.pos[1] + 0.5 for p in model.pedestrians if not p.completed]
    ped_y = [model.height - 1 - p.pos[0] + 0.5 for p in model.pedestrians if not p.completed]
    if ped_x:
        ax.scatter(ped_x, ped_y, c='blue', s=50, label='Peatón')

    veh_x = [v.pos[1] + 0.5 for v in model.vehicles if v.active and not v.completed]
    veh_y = [model.height - 1 - v.pos[0] + 0.5 for v in model.vehicles if v.active and not v.completed]
    if veh_x:
        ax.scatter(veh_x, veh_y, c='red', s=50, marker='s', label='Vehículo')

    if hasattr(model, 'authority'):
        auth_color = 'lime' if not model.authority.completed else 'gray'
        ax.scatter(model.authority.pos[1] + 0.5,
                  model.height - 1 - model.authority.pos[0] + 0.5,
                  c=auth_color, s=120, marker='s', label='Autoridad', zorder=5)

    ax.set_xlim(0, model.width)
    ax.set_ylim(0, model.height)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(loc='upper right')

params = {
    'width': 26,
    'height': 26,
    'obs_density': 0.04,
    'weight': 1.3,
    'max_pedestrians': 12,
    'max_vehicles': 12,
    'green_duration': 12,
    'building_size': 8,
    'steps': 55
}

model = CrossroadModel(params)
fig, ax = plt.subplots(figsize=(10, 8))
animation = ap.animate(model, fig, ax, plot, steps=params['steps'], seed=42)
HTML(animation.to_jshtml())

Connected to Unity server
